In [1]:
import gurobipy as gp
import numpy as np

In [5]:
def cria_instancia(nome_arquivo):
    with open(nome_arquivo, 'r') as f:
        vrp = f.readlines()  #Lendo a instância
    
    #Descobindo índices para fatiamento futuro
    index_nos = vrp.index('NODE_COORD_SECTION \n')  
    index_demanda = vrp.index('DEMAND_SECTION \n')
    
    #número de ônibus
    qtd_onibus = int(vrp[1].split(':')[2][1])
    
    #capacidade
    capacidade = int(vrp[5].split(':')[1].strip())
    
    #Coordenada das "lojas"
    coord_variaveis = list(map(lambda x: np.array(x.strip().split(' ')[1:3], dtype = np.float32) , vrp[index_nos+1:index_demanda]))

    qtd_variaveis = len(vrp[index_nos+1:index_demanda]) -1 #desconsidera o depósito como variável
    
    #Cria dicionário com as distâncias euclidianas entre os nós
    dict_distancias = {f'c{i},{j}': np.round(np.linalg.norm(coord_variaveis[i]-coord_variaveis[j]),2) for i in range(qtd_variaveis+1) for j in range(qtd_variaveis+1)}
    
    #Armazena as demandas de cada cliente
    vetor_demandas = list(map(lambda lista: float(lista.split(' ')[1]),vrp[index_demanda+2:index_demanda+2+qtd_variaveis]))
    
    #Cria rótulos para as variaveis
    origens = [i for i in range(qtd_variaveis+1)] #0 corresponde ao depósito
    destinos = [i for i in range(qtd_variaveis+1)]
    onibus = [j+1 for j in range(qtd_onibus)]
    
    #Dicionário com as demandas
    dict_demandas = {i+1: vetor_demandas[i] for i in range(qtd_variaveis)}

    return capacidade,dict_distancias, dict_demandas, origens, destinos, onibus, coord_variaveis

In [3]:
def cria_modelo(capacidade, dict_distancias, dict_demandas, origens, destinos, onibus):
    #Criando Modelo
    m = gp.Model()

    #Criando variáveis
    x = m.addVars(onibus,origens,destinos, vtype = gp.GRB.BINARY)
    u = m.addVars(origens[1:])

    # Função Objetivo
    m.setObjective(
        gp.quicksum(dict_distancias[f'c{i},{j}']*x[k,i,j] for k in onibus for j in destinos for i in origens if i!=j ),
        sense = gp.GRB.MINIMIZE
    )

    #Adicionando restrições

    #Restrição de capacidade

    m.addConstrs(
        gp.quicksum(x[k,i,j]*dict_demandas[i] for i in origens[1:] for j in destinos if j!=i) <= capacidade
        for k in onibus
    )

    #Conservação de fluxo

    m.addConstrs(
        gp.quicksum((x[k,i,p] - x[k,p,i])   for i in origens if p!=i) == 0
        for k in onibus for p in origens
    )

    # Cada nó é visitado uma vez, exceto a origem
    m.addConstrs(
        gp.quicksum(x[k,i,j] for k in onibus for i in origens if i!=j) ==1
        for j in destinos[1:]
    )

    # Cada veículo deixa o deposito

    m.addConstrs(
        gp.quicksum(x[k,0,j] for j in destinos[1:])==1
        for k in onibus
    )

    # Eliminação de sub-rotas Miller-Tucler

    m.addConstrs(u[i] <=capacidade  for i in origens[1:])
    m.addConstrs(dict_demandas[i] <= u[i] for i in origens[1:])

    m.addConstrs((u[j]-u[i]-dict_demandas[j] + capacidade*(1-gp.quicksum(x[k,i,j] for k in onibus))) >=0
                 for i in origens[1:]
                 for j in destinos[1:] if i!=j)
    
    return m, x

In [6]:
capacidade, dict_distancias, dict_demandas, origens, destinos, onibus, coord = cria_instancia(nome_arquivo = 'Lista2.txt')

In [7]:
m, x = cria_modelo(capacidade, dict_distancias, dict_demandas, origens, destinos, onibus)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-01


In [7]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 5 3500U with Radeon Vega Mobile Gfx, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 159 rows, 373 columns and 1760 nonzeros
Model fingerprint: 0x632ef123
Variable types: 10 continuous, 363 integer (363 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [3e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Presolve removed 20 rows and 33 columns
Presolve time: 0.03s
Presolved: 139 rows, 340 columns, 1704 nonzeros
Variable types: 10 continuous, 330 integer (330 binary)

Root relaxation: objective 3.999346e+02, 71 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  399.93460    0   23          -  399.93460      -

In [8]:
import pandas as pd

In [9]:
for k in onibus:
    matriz_adjacente = []
    for i in origens:
        lista_i = []
        for j in destinos:
            lista_i.append(round(x[k,i,j].X))
        matriz_adjacente.append(lista_i)
    df = pd.DataFrame(matriz_adjacente, index = origens, columns = destinos)
    df.to_csv(f'matriz_adjacente_{k}.csv')

In [11]:
df = pd.read_csv('matriz_adjacente_1.csv', header = 0, index_col = 0)
df

,0,1,2,3,4,5,6,7,8,9,10
0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0
